# 拍點音符紀錄器 for 和弦判斷
- 附點< dots >, 多連音< Tuplet >, 各種拍子、變拍< TimeSig >已處理


In [1]:
import numpy as np
from decimal import *

def convert(tu):
    origin = [0,0,0,0,0,0,0,0,0,0,0,0]
    for i in tu:
        origin[i%12]=1
    return tuple(origin)

# chord = {
#     convert((0,4,7)):'Major',convert((0,3,7)):'minor',
#     convert((0,4,8)):'aug',convert((0,3,6)):'dim',
#     convert((0,4,7,11)):'Major7',convert((0,4,7,10)):'dom7',
#     convert((0,3,7,10)):'minor7',convert((0,3,6,10)):'m7flat5',
#     convert((0,2,7)):'2',convert((0,5,7)):'4',
#     convert((0,2,4,7,10)):'9'
# }
chord = {
    convert((0,4,7)):'C',convert((2,5,9)):'Dm',convert((4,7,11)):'Em',
    convert((0,5,9)):'F',convert((2,7,11)):'G',convert((0,4,9)):'A',
}

chord_np = np.array(chord.keys())

key = {
    0:'C',1:'C-sharp',2:'D',3:'D-sharp',4:'E',5:'F',6:'F-sharp',7:'G',8:'G-sharp',9:'A',10:'A-sharp',11:'B'
} # pitch%12

time = {
    'measure':None,
    'whole':Decimal(1),
    'half':Decimal(1)/2,
    'quarter':Decimal(1)/4,
    'eighth':Decimal(1)/8,
    '16th':Decimal(1)/16,
    '32nd':Decimal(1)/32,
    '64th':Decimal(1)/64,
    '128th':Decimal(1)/128
}

def ifDots(tag):
    if not tag.find('dots'):
        multi = 1
    else:
        dots = tag.find('dots').text
        if dots == '1':
            multi = 1.5
        elif dots == '2':
            multi = 1.75
        elif dots == '3':
            multi = 1.875
    return Decimal(multi)

In [2]:
from bs4 import BeautifulSoup as bs
import sys

#Basket Case是C小調(聽起來像Eb大調)
f = open('C:/Users/BigData/Desktop/mscx/ABBA_Dancing_Queen_CHT.mscx','r')
music = bs(f.read(),'xml')
f.close()
staff1TimeSig = music.select_one('Score > Staff:nth-of-type(1) TimeSig')
sigN = Decimal(staff1TimeSig.find('sigN').text)
sigD = Decimal(staff1TimeSig.find('sigD').text)
time['measure'] = sigN/sigD
l = []
dic = {}
for staff in music.select('Score > Staff'):
    time_acc = 0
    tupletID = ''
    tupletRatio = 1
    for tag in staff.find_all(['Rest','Chord','Tuplet','TimeSig']):

        #Additional <track> in every measure is ignored
        if (tag.name == 'Chord' or tag.name == 'Rest') and not tag.find('track'):
            multi = ifDots(tag)
            if tag.find('Tuplet') and tag.find('Tuplet').text == tupletID:
                multi *= tupletRatio
            duTime = Decimal(time[tag.select('durationType')[0].text])
            time_acc += duTime * multi
            if tag.name == 'Rest':
                l.append((time_acc, 0))
            else:
                pitches = tag.select('pitch')
                
                #目前音高為手動修改
                addlist = [time_acc] + [int(a.text)-3 for a in pitches]
                l.append(tuple(addlist))
        
        #Only take <Tuplet> with id to avoid children
        #<Tuplet> inside <Chord> or <Rest>
        elif tag.name == 'Tuplet' and tag.get('id'):
            tupletID = tag['id']
            normalNotes = Decimal(tag.find('normalNotes').text)
            actualNotes = Decimal(tag.find('actualNotes').text)
            tupletRatio = normalNotes / actualNotes
        
        elif tag.name == 'TimeSig':
            sigN = Decimal(tag.find('sigN').text)
            sigD = Decimal(tag.find('sigD').text)
            if sigN/sigD != time['measure']:
                time['measure'] = sigN/sigD
    dic.update({'staff_'+staff.get('id') : l})
    l = []

In [5]:
for i, staff in enumerate(dic, 1):
    print '[Staff #' + str(i) + ']'
    print '-------------------------'
    for e in dic[staff]:
        print e
    print '-------------------------'

[Staff #1]
-------------------------
(Decimal('1'), 0)
(Decimal('2'), 0)
(Decimal('3'), 0)
(Decimal('4'), 0)
(Decimal('5'), 0)
(Decimal('6'), 0)
(Decimal('7'), 0)
(Decimal('8'), 0)
(Decimal('9'), 0)
(Decimal('9.5'), 0)
(Decimal('9.75'), 0)
(Decimal('10.00'), 68)
(Decimal('10.0625'), 70)
(Decimal('10.1250'), 68)
(Decimal('10.5000'), 66)
(Decimal('10.8750'), 66)
(Decimal('11.0000'), 0)
(Decimal('11.2500'), 70)
(Decimal('11.3750'), 68)
(Decimal('11.5000'), 68)
(Decimal('11.7500'), 68)
(Decimal('11.8125'), 68)
(Decimal('11.8750'), 0)
(Decimal('12.0000'), 0)
(Decimal('12.2500'), 70)
(Decimal('12.3750'), 68)
(Decimal('12.5000'), 68)
(Decimal('12.6250'), 68)
(Decimal('13.0000'), 70)
(Decimal('13.1875'), 66)
(Decimal('13.2500'), 68)
(Decimal('13.3750'), 68)
(Decimal('13.5000'), 65)
(Decimal('13.6875'), 66)
(Decimal('13.7500'), 68)
(Decimal('13.8750'), 68)
(Decimal('14.0000'), 65)
(Decimal('14.0625'), 66)
(Decimal('14.1250'), 65)
(Decimal('14.5000'), 63)
(Decimal('14.6250'), 70)
(Decimal('14.68

# 以下未修改

In [169]:
# 第一步，判斷每個staff，有幾成的比例，使用和弦   by Henry
# 1-1. 目前預設，只要大於70%，就紀錄該staff
# 1-2. pitch只要同時有2音以上，就當作和弦

pitch_count = 0
percent = 0.7
dominant_percent_staff_list = []

for staff in dic:
    for tempo_pitch in dic[staff]:
        if len(tempo_pitch) > 3 :
            pitch_count += 1
    if float(pitch_count) / float(len(dic[staff])) >= percent:
        dominant_percent_staff_list.append(staff)
print dominant_percent_staff_list

#測試區

# for staff in dominant_percent_staff_list:
#     print staff
# print dic['staff_4']
# print type(dic)
# for staff in dic:
#     print staff
# for tempo_pitch in dic[staff]:
#     print tempo_pitch
#     print len(tempo_pitch)
# print pitch_count
# print type(pitch_count)
# print len(dic['staff_2'])
# print type(len(dic['staff_2']))
# if float(pitch_count) / float(len(dic['staff_2'])) >= 0.3:
#     dominant_percent_staff_list.append('staff_2')
#     print dominant_percent_staff_list
print type(dic[staff][-12][1:])
print type(chord_np)
print type(convert_chord(tuple(now)))

[u'staff_2', u'staff_3']
<type 'tuple'>
<type 'numpy.ndarray'>
<type 'tuple'>


In [157]:
import numpy as np
import math
def nearest(nparray, pt):
    n = [math.sqrt(sum((i-pt)**2)) for i in nparray]
    mindis = min(n)
    idx = n.index(mindis)
    return nparray[idx]


In [170]:
chord_dic = {}
pitch_count = 0
percent = 0.7
for staff in dic:
    chord_gogo = []
    chord_gogogo = []
    count = 0
    begin = 0
    
    for tempo_pitch in dic[staff]:
        if len(tempo_pitch) > 3 :
            pitch_count += 1
    if float(pitch_count) / float(len(dic[staff])) >= percent:
        
        while(count < dic[staff][-1][0]):
            while(count < dic[staff][begin][0]):
                count += 1.0/2
                if dic[staff][begin][1] == 0:
                    chord_gogo.append(0)
                else:
                    now = [pit%12 for pit in dic[staff][begin][1:]]
                    closest = nearest(chord_np,convert_chord(tuple(now)))  
                    chord_gogo.append(chord[tuple(closest)])

            begin += 1
        chord_dic.update({staff : chord_gogo})

In [177]:
print len(chord_dic['staff_3'])

450


In [116]:
for ch in chord_dic['staff_2']:

SyntaxError: invalid syntax (<ipython-input-116-8430e92f8c05>, line 1)

In [147]:
c = []
for i in range(1,len(chord_dic['staff_1'])+1):
    if i%8 == 0:
        c.append(chord_dic['staff_1'][i-1])
        print c
        c = []
    else:
        c.append(chord_dic['staff_1'][i-1])


[0, 0, 0, 0, 'C', 'C', 'G', 'G']
['A', 'A', 'Em', 'Em', 'F', 'F', 'C', 'C']
['G', 'G', 'G', 'G', 'C', 'C', 'G', 'G']
['A', 'A', 'Em', 'Em', 'F', 'F', 'C', 'C']
['G', 'G', 'G', 'G', 'F', 'F', 'G', 'G']
['C', 'C', 'C', 'C', 'F', 'F', 'G', 'G']
['C', 'C', 'C', 'C', 'F', 'F', 'G', 'G']
['C', 'F', 'A', 'A', 'F', 'F', 'G', 'G']
['C', 'G', 'A', 'G', 'C', 'G', 'A', 'G']
['C', 'C', 'G', 'G', 'A', 'A', 'Em', 'Em']
[0, 0, 0, 'F', 'A', 0, 'F', 0]
[0, 0, 'C', 'C', 'G', 'G', 'G', 'G']
['C', 'C', 'G', 'G', 'A', 'A', 'Em', 'Em']
['F', 0, 'G', 'C', 'A', 0, 'F', 0]
[0, 0, 'C', 'C', 'G', 'G', 'G', 'G']
['F', 'F', 'G', 'G', 'C', 'C', 'C', 'C']
['F', 'F', 'G', 'G', 'C', 'C', 'C', 'C']
['F', 'F', 'G', 'G', 'C', 'F', 'A', 'A']
['F', 'F', 'G', 'G', 'C', 'G', 'A', 'G']
['C', 'G', 'A', 'G', 'C', 'G', 'A', 'G']
['C', 'G', 'A', 'G', 'F', 'F', 'F', 'F']
['G', 'G', 'G', 0, 'C', 'C', 'G', 'G']
['A', 'A', 'Em', 'Em', 0, 0, 'F', 'C']
[0, 0, 0, 0, 0, 0, 'C', 'C']
['G', 'G', 'G', 'G', 'C', 'C', 'G', 'G']
['A', 'A', 'Em'

In [70]:
chord_gogo = []
count = 0
begin = 0
while(count<l2[-1][0]):
    while(count < l2[begin][0]):
        count += 1.0/2
        if l2[begin][1] == 0:
            chord_gogo.append(0)
        else:
            chord_gogo.append(key[l2[begin][1]%12])
    begin += 1

[5, 1, 3]


In [ ]:
now = [pit%12 for pit in dic[staff][begin][1:]]
                for pitch in now:
                    poss_key = key[pitch]
                    det = [(n-pitch)%12 for n in now]
                    vec = convert_chord(tuple(det))
                    poss_chord = nearest(chord_np,vec)[0]